In [1]:
import pandas as pd
from folium import plugins, Map, CircleMarker
from folium.plugins import HeatMap
import geopandas as gpd
import folium
import geopy
import asyncio
from shapely.geometry import Polygon, Point
from pandas.io.json import json_normalize
import shapely.geometry



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
dest_epsg = '32633'

In [3]:
#Use the shape file to get additional data
sidewalks =gpd.read_file('../data/shape/Sidewalks.shx')
sidewalks['ID'] = sidewalks.index
sidewalks_hazards = gpd.read_file('../data/shape/Sidewalk_Hazards.shx')
ramps_0 = gpd.read_file('../data/Ramps_0_with_score.csv')
sidewalks = sidewalks[['ID','geometry','DISTRICT']]
sidewalks = gpd.GeoDataFrame(sidewalks)
sidewalks = sidewalks.to_crs(dest_epsg)
sidewalks_hazards = sidewalks_hazards[['GlobalID', 'geometry', 'Type']]
sidewalks_hazards = sidewalks_hazards.to_crs(dest_epsg)

ramps_0['geometry'] =  gpd.points_from_xy(ramps_0.Longitude, ramps_0.Latitude, crs="EPSG:4326")
ramps_0 = gpd.GeoDataFrame(ramps_0)
ramps_0 = ramps_0.to_crs(dest_epsg)
ramps_0 = ramps_0[['FID', 'geometry', 'score_1']]


sidewalks_full = gpd.GeoDataFrame(sidewalks)
sidewalks_hazards_full = gpd.GeoDataFrame(sidewalks_hazards)
ramps_0_full = gpd.GeoDataFrame(ramps_0)

In [4]:
ramps_0 = gpd.GeoDataFrame(ramps_0_full)
ramps_0

,FID,geometry,score_1
0,1176,POINT (-5525947.827 9520222.277),82.4
1,1211,POINT (-5526075.307 9520496.452),92.8
2,1212,POINT (-5526058.600 9520505.601),66.34115385
3,1213,POINT (-5526048.113 9520487.515),95.975
4,1214,POINT (-5525926.752 9520235.683),87.75
...,...,...,...
26154,26157,POINT (-5530854.625 9517963.749),86.4
26155,26158,POINT (-5530832.405 9518275.111),92.65
26156,26159,POINT (-5530111.833 9518070.376),95.97
26157,26160,POINT (-5539128.679 9527785.272),64.15


In [6]:
sidewalks = gpd.GeoDataFrame(sidewalks_full)
ramps_0 = gpd.GeoDataFrame(ramps_0_full)
sidewalks = sidewalks
ramps_0['Sidewalk_ID'] = ramps_0['geometry'].apply(lambda x: sidewalks[x.buffer(10).intersects(sidewalks['geometry'])].index)
# sidewalks_hazards['Sidewalk_distance'] = sidewalks_hazards.apply(lambda x: x['geometry'].distance(sidewalks['geometry'].iloc[x['Sidewalk_ID']]), axis = 1)
ramps_0['Sidewalk_ID'] = ramps_0['Sidewalk_ID'].apply(lambda x: -1 if len(x)==0 else x[0])
ramps_0 = ramps_0[ramps_0['Sidewalk_ID']!=-1]

ramps_0['ID'] = ramps_0['Sidewalk_ID']

merged = ramps_0.join(sidewalks, on = 'ID', rsuffix = 'x')
merged = merged[['geometry', 'geometryx', 'ID','DISTRICT']]

 
merged.to_csv('../data/merged_sidewalks_and_ramps.csv')

In [7]:
merged = ramps_0.join(sidewalks, on = 'ID', rsuffix = 'x')
merged = merged[['FID', 'geometry', 'geometryx', 'ID', 'score_1']]

 
merged.to_csv('../data/merged_sidewalks_and_ramps.csv')

In [12]:
merged['score_1'] = merged['score_1'].astype(float)

In [13]:
merged['bad ramps'] = merged['score_1'].apply(lambda x: 1 if x<= 20 else 0)


In [33]:
cnt = merged.groupby('ID')['bad ramps'].sum()

In [34]:
cnt = cnt.reset_index()
cnt = cnt.join(sidewalks, on  ='ID',rsuffix = 'r')

In [35]:
cnt = gpd.GeoDataFrame(cnt)

In [36]:
cnt

,ID,bad ramps,IDr,geometry,DISTRICT
0,0,0,0,"POLYGON ((-5521066.633 9520273.075, -5521068.6...",CHARLESTOWN
1,1,0,1,"POLYGON ((-5538366.668 9521995.632, -5538369.8...",SOUTH DORCHESTER
2,2,0,2,"POLYGON ((-5538262.776 9522047.158, -5538262.8...",SOUTH DORCHESTER
3,3,0,3,"POLYGON ((-5538231.333 9522025.847, -5538234.6...",SOUTH DORCHESTER
4,4,1,4,"POLYGON ((-5538130.175 9522070.014, -5538130.3...",SOUTH DORCHESTER
...,...,...,...,...,...
14156,23504,0,23504,"MULTIPOLYGON (((-5535860.183 9521820.337, -553...",SOUTH DORCHESTER
14157,23505,0,23505,"POLYGON ((-5536076.442 9520545.054, -5536080.6...",SOUTH DORCHESTER
14158,23506,0,23506,"POLYGON ((-5536091.656 9520515.510, -5536091.7...",SOUTH DORCHESTER
14159,23507,0,23507,"POLYGON ((-5536078.128 9520453.066, -5536081.0...",SOUTH DORCHESTER


In [42]:
import folium
import pandas as pd

# Load data
cnt = cnt.to_crs(epsg=4326)
ramps_0 = pd.read_csv('../data/Ramps_0_with_score.csv')
sidewalks_hazards = pd.read_csv("../data/preprocessed_sidewalks_hazards.csv")

ramps_0['x'] = ramps_0['Longitude'].astype(float)
ramps_0['y'] = ramps_0['Latitude'].astype(float)
map = folium.Map(location=[42.2995, -71.0649], zoom_start=15)

# Add sidewalks layer
# merged_sidewalk_hazards['centroids']= gpd.GeoSeries.from_wkt(merged_sidewalk_hazards['centroids'])
# merged_sidewalk_hazards = gpd.GeoDataFrame(merged_sidewalk_hazards)
# merged_sidewalk_hazards= merged_sidewalk_hazards.set_geometry('centroids')
# merged_sidewalk_hazards.crs = dest_epsg
# merged_sidewalk_hazards = merged_sidewalk_hazards.to_crs('4326')

def color_cluster_sidewalks(cnt_):
    # print(cnt_)
 
    cnt1 = cnt_['properties']['bad ramps']
    if cnt1 > 5:
        return  {'fillColor': 'red', 'color': 'red'}
    elif cnt1 >= 2:
        return  {'fillColor': 'orange', 'color': 'orange'}
    elif cnt1 > 0:
        return {'fillColor': 'yellow', 'color': 'yellow'}
    else:
        return {'fillColor': 'green', 'color': 'green'}
    
red_layer = folium.FeatureGroup(name='Bad Ramps more than 5(RED)')
orange_layer = folium.FeatureGroup(name = 'Bad Ramps more than 2 less than 5(ORANGE)')
yellow_layer = folium.FeatureGroup(name='Bad Ramps more than 0 less than 2(YELLOW)')
green_layer = folium.FeatureGroup(name='No Bad Ramps(GREEN)')



# def convert(row):
#     t = shapely.geometry.mapping(row['geometry'])
#     return {
#        "type": "Feature",
#   "geometry": {
#     "type": row['geometry'].geom_type,
#     "coordinates": t['coordinates']
#   },
#   "properties": {
#     "bad ramps": row['bad ramps']
#   }
#     }
# for index,row in cnt.iterrows():

#     if row['geometry']== None:
#         continue
#     b = folium.GeoJson(convert(row), style_function = color_cluster_sidewalks)
#     # b.add_child(folium.Popup("Index                       : " + str(index) + "<br>Score: " + str(row['cnt'])
#     #                      + "<br><br>Fixed pinch point     : " + str(row['Fixed pinch point <36” sidewalk width'])
#     #                      + "<br><br>Trip due to tree roots: " + str(row['Trip hazards due to tree roots'])
#     #                      + "<br><br>Trip not to teee roots: " + str(row['Trip hazards not due to tree roots']),
#     #                      max_width=400))
#     if row['bad ramps'] > 5:
#         b.add_to(red_layer)
#     elif row['bad ramps'] > 2:
#         b.add_to(orange_layer)
#     elif row['bad ramps'] > 0:
#         b.add_to(yellow_layer)
#     else:
#         b.add_to(green_layer)

sidewalks_ = folium.GeoJson(cnt[cnt['bad ramps'] > 5], style_function = color_cluster_sidewalks)
sidewalks_.add_to(red_layer)
sidewalks_ = folium.GeoJson(cnt[(cnt['bad ramps'] <= 5) & (cnt['bad ramps'] >= 2)], style_function = color_cluster_sidewalks)
sidewalks_.add_to(orange_layer)
sidewalks_ = folium.GeoJson(cnt[(cnt['bad ramps'] <2) & (cnt['bad ramps'] >0 )], style_function = color_cluster_sidewalks)
sidewalks_.add_to(yellow_layer)
sidewalks_ = folium.GeoJson(cnt[cnt['bad ramps']==0], style_function = color_cluster_sidewalks)
sidewalks_.add_to(green_layer)

red_layer.add_to(map)
orange_layer.add_to(map)
yellow_layer.add_to(map)
green_layer.add_to(map)
def color_cluster(index):
    if index <= 20:
        return 'red'
    elif index <= 50:
        return 'green'
    elif index <= 80: 
        return 'blue'
    else:
        return 'cyan'

hazards_layer = folium.FeatureGroup(name='Ramps')
for point in range(0, len(sidewalks_hazards)):
    folium.CircleMarker(
        [ramps_0.iloc[point]['y'], sidewalks_hazards.iloc[point]['x']] ,
        color=color_cluster(ramps_0.iloc[point]['score_1']),
        fill=True,
        fill_color=color_cluster(ramps_0.iloc[point]['score_1'])
    ).add_to(hazards_layer)


# centroid_layer = folium.FeatureGroup(name='centroids')
# for point in range(0, len(merged_sidewalk_hazards)):
#     icon = folium.Icon(icon = 'exclamation-sign', color = 'red')
#     folium.Marker([merged_sidewalk_hazards.iloc[point]['centroids'].y, merged_sidewalk_hazards.iloc[point]['centroids'].x], icon = icon).add_to(centroid_layer)
hazards_layer.add_to(map)
# centroid_layer.add_to(map)

folium.LayerControl().add_to(map)
map.save("../maps/sidewalks/sidewalks_with_ramps_score.html")